<a href="https://colab.research.google.com/github/zhuchunlin1995/Deep-Learning/blob/master/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#download the CIFAR10 dataset for image classification.
import torch, torchvision
import numpy as np
import random

transform = torchvision.transforms.Compose( [torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)

#preprocess trainset.
dataset = []
#flat dataset
for data in trainset:
  tmp = []
  tmp.append(data[0].numpy().ravel())
  tmp.append(data[1])
  dataset.append(tmp)


random.shuffle(dataset)
#separate features and label
dataset_X = [item[0].tolist() for item in dataset]
dataset_Y = [item[1] for item in dataset]

#combined training sample:
training_set = dataset[5000:]

#split dataset into training set and validation set and format the dataset

valiset_X = np.asarray(dataset_X[:5000])
valiset_Y = dataset_Y[:5000]


trainset_X = np.asarray(dataset_X[5000:]) #shape (num of training data, num of features)
trainset_Y = dataset_Y[5000:]



Files already downloaded and verified
Files already downloaded and verified


In [2]:
class NeuralNetwork:
  
  def __init__(self, layer_dimensions):
    #initialize weights and bias
    self.parameters = {}
    self.cache = {}
    self.num_layers = len(layer_dimensions) - 1
    for l in range (1, self.num_layers + 1):
      eps = np.sqrt(2.0 / (layer_dimensions[l] + layer_dimensions[l - 1]))
      self.parameters["W" + str(l)] = np.random.randn(layer_dimensions[l], layer_dimensions[l - 1]) * eps #shape (features, neurons)
      self.parameters["b" + str(l)] = np.zeros((layer_dimensions[l], 1)) + 0.01 #shape (1,n)
  
  def affineForward(self, A, W, b, layer):
    forward = np.dot(W, A) + b
    self.cache["Z" + str(layer)] = forward
    return forward
    
  def activationForward(self, A, layer):
    # use relu function as activation function
    A = np.maximum(0, A)
    self.cache["A" + str(layer)] = A
    return A
  
  def _softmax(self, AL):
    num_data = AL.shape[1]
    result = np.zeros(AL.shape)
    # matrix of 1s
    for n in range(num_data):
      curr = AL[:,n]
      e_curr = np.exp(curr - np.max(curr))
      result[:,n] = e_curr/np.sum(e_curr)
    return result
  
  def costFunction(self, AL, y):
    AL_softmax = self._softmax(AL)
    # calculate cross entropy loss
    n = len(y)
    log_likelyhood = -np.log(AL_softmax[y, range(n)]++1e-9)
    loss = np.sum(log_likelyhood) / n
    return loss
  
  
  def affineBackward(self, dL_dZ, layer, rate):
    A = self.cache["A" + str(layer - 1)]
    dL_dW = np.dot(dL_dZ ,A.transpose())
    W = self.parameters["W" + str(layer)]
    self.parameters["W" + str(layer)] =  self.parameters["W" + str(layer)] - rate * dL_dW
    self.parameters["b" + str(layer)] =  self.parameters["b" + str(layer)] - rate * np.dot(dL_dZ, np.ones((dL_dZ.shape[1],1)))
    return np.dot(W.transpose(), dL_dZ)
     
  
  
  def activationBackward(self, dL_dA, layer):
    Z = self.cache["Z" + str(layer - 1)]
    Z[Z <= 0] = 0
    Z[Z > 1] = 1
    # multiplication
    return dL_dA*Z
    
  def backPropagation(self, y):
    AL = self.cache["Z" + str(self.num_layers)]
    grad = self._softmax(AL)
    n = len(y)
    grad[y, range(n)] -= 1
    grad = grad / n
    return grad
  
  def forward(self, X_train):
    self.cache["A0"] = X_train
    for layer in range(1, self.num_layers):
      Z = self.affineForward(self.cache["A"+ str(layer - 1)], self.parameters["W" + str(layer)], self.parameters["b" + str(layer)], layer)
      A = self.activationForward(Z, layer)
    Z = self.affineForward(self.cache["A" + str(self.num_layers - 1)], self.parameters["W" + str(self.num_layers)], self.parameters["b" + str(self.num_layers)], self.num_layers)
    return Z
  
  def predict(self, X_test):
    Z = self.forward(X_test)
    Z = self._softmax(Z)
    Y_Pred = np.argmax(Z, axis=0)
    return Y_Pred

  def validate(self, valiset_X, valiset_Y):
    Y_pred = self.predict(valiset_X)
    correct = 0
    for i in range(0, len(Y_pred)):
      if (Y_pred[i] == valiset_Y[i]):
        correct += 1
    return correct / len(valiset_Y)
  
  def train(self, trainset, X_val, Y_val, iters, alpha, batch_size):
    
    for iter in range(iters):
      print("Iteration: " + str(iter))
      random.shuffle(trainset)
      X_train = [item[0].tolist() for item in trainset]
      Y_train = [item[1] for item in trainset]
      l = 0
      for i in range(0, len(X_train)-1, batch_size):
        X_train_mini = np.transpose(X_train[i : i + batch_size])
        Y_train_mini = np.transpose(Y_train[i : i + batch_size])
        Z = self.forward(X_train_mini)
        loss = self.costFunction(Z,Y_train_mini)
        dL_dZ = self.backPropagation(Y_train_mini)
        for layer in range(self.num_layers, 0, -1):
          dL_dA = self.affineBackward(dL_dZ,layer, alpha)
          if layer > 1:
            dL_dZ = self.activationBackward(dL_dA, layer)
        l = loss
      train_accuracy = self.validate(trainset_X.transpose(), trainset_Y)
      validate_accuracy = self.validate(valiset_X.transpose(), valiset_Y)
      
        
      print("training loss is ", l)
      print("training accuracy is ", train_accuracy)
      print("validation accuracy is ", validate_accuracy)
      print(" ")

if __name__ == "__main__":
  layer_dimensions = [3072, 600, 500, 10]
  nn = NeuralNetwork(layer_dimensions)
  nn.train(training_set, [], [], 15, 0.01, 50)
 
  

Iteration: 0
training loss is  1.7764639465987546
training accuracy is  0.40275555555555553
validation accuracy is  0.3888
 
Iteration: 1
training loss is  1.747398341337623
training accuracy is  0.4446
validation accuracy is  0.4332
 
Iteration: 2
training loss is  1.6867776195266115
training accuracy is  0.4696
validation accuracy is  0.4464
 
Iteration: 3
training loss is  1.3833475200976517
training accuracy is  0.4975777777777778
validation accuracy is  0.467
 
Iteration: 4
training loss is  1.3868560710314843
training accuracy is  0.5142222222222222
validation accuracy is  0.4802
 
Iteration: 5
training loss is  1.3291142033425338
training accuracy is  0.5268666666666667
validation accuracy is  0.4924
 
Iteration: 6
training loss is  1.281938767161841
training accuracy is  0.5373555555555556
validation accuracy is  0.4852
 
Iteration: 7
training loss is  1.500319052193226
training accuracy is  0.5626666666666666
validation accuracy is  0.5008
 
Iteration: 8
training loss is  1.19